## Part2. Compare Taste Predictors 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
PWD = os.path.abspath('.')+'/'
print(PWD)

/home/hy/Documents/Project/bitterants/


In [2]:
bs_train_path = os.path.join(PWD, 'dataset/taste_dataset/bitter_sweet/bs_train.csv')
bn_train_path = os.path.join(PWD, 'dataset/taste_dataset/bitter_nonbitter/bn_train.csv')
bs_test_path = os.path.join(PWD, 'dataset/taste_dataset/bitter_sweet/bs_test.csv')
bn_test_path = os.path.join(PWD, 'dataset/taste_dataset/bitter_nonbitter/bn_test.csv')

train other predictors -- 1. CNN and MLP in https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005

In [3]:

from hgnn.trainers import train_mlp_cnn
from contrast_predictors.CNN import *
from contrast_predictors.MLP import *

bs_train = loader(bs_train_path) 
bn_train = loader(bn_train_path) 
bs_test = loader(bs_test_path) 
bn_test = loader(bn_test_path) 


model = BoCNN()
bn_train_imgs_dir = os.path.join(PWD, 'dataset/taste_dataset/bitter_nonbitter/bn_train_imgs/') 
if not os.path.exists(bn_train_imgs_dir):
    os.mkdir(bn_train_imgs_dir)
bn_cnn_train = cnn_prepare(bn_train, bn_train_imgs_dir, data_aug=True)
all = all_batchsize(bn_cnn_train, batchsize=int(len(bn_cnn_train)/16))
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bn_cnn.pth',)

model = BoCNN()
bs_train_imgs_dir = os.path.join(PWD, 'dataset/taste_dataset/bitter_sweet/bs_train_imgs/') 
if not os.path.exists(bs_train_imgs_dir):
    os.mkdir(bs_train_imgs_dir)
bs_cnn_train = cnn_prepare(bs_train, bs_train_imgs_dir, data_aug=True)
print(len(bs_cnn_train))
all = all_batchsize(bs_cnn_train, batchsize=int(len(bs_cnn_train)/16))
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bs_cnn.pth',)

bn_ds_mlp_train = Descriptors_bitter_nonbitter(bn_train)
all = all_batchsize(bn_ds_mlp_train, batchsize=int(len(bn_ds_mlp_train)/16))
model = BoMLP(n_feats=21)
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bn_ds_mlp.pth')


bs_ds_mlp_train = Descriptors_bitter_sweet(bs_train)
all = all_batchsize(bs_ds_mlp_train, batchsize=int(len(bs_ds_mlp_train)/16))
model = BoMLP(n_feats=17)
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bs_ds_mlp.pth')


bn_rdkfp_mlp_train = RDKFP(bn_train)
all = all_batchsize(bn_rdkfp_mlp_train, batchsize=int(len(bn_rdkfp_mlp_train)/16))
model = BoMLP(n_feats=2048)
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bn_rdkfp_mlp.pth')


bs_rdkfp_mlp_train = RDKFP(bs_train)
all = all_batchsize(bs_rdkfp_mlp_train, batchsize=int(len(bs_rdkfp_mlp_train)/16))
model = BoMLP(n_feats=2048)
train_mlp_cnn.train_bi_classify_all(model, all=all, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/', save_name='bs_rdkfp_mlp.pth')


loss: 0.6149703492136562 ACC: 0.6816810640340051 AUC: 0.6412951068366473
loss: 0.5668444037437439 ACC: 0.7080899492664197 AUC: 0.6761815896176262
loss: 0.5040962590890772 ACC: 0.7534896476072946 AUC: 0.7188550490269546
loss: 0.4366242570035598 ACC: 0.7955710955710957 AUC: 0.7744626481852404
loss: 0.39052246423328624 ACC: 0.8273961332784863 AUC: 0.815698222207054
loss: 0.32721036321976604 ACC: 0.8589195118606885 AUC: 0.8471779072241037
loss: 0.26861845307490406 ACC: 0.8932949403537641 AUC: 0.8811212261181662
loss: 0.2528698277824065 ACC: 0.9055121349238998 AUC: 0.8950606732539194
loss: 0.17065640538930893 ACC: 0.9351432880844647 AUC: 0.929697514912665
Early stopping
5701
loss: 0.6086662411689758 ACC: 0.6962656972901522 AUC: 0.6823065735769941
loss: 0.622581190922681 ACC: 0.655056179775281 AUC: 0.6493734486821753
loss: 0.5256393621949589 ACC: 0.7407138136153338 AUC: 0.7296872714191608
loss: 0.523124650997274 ACC: 0.7477858559153999 AUC: 0.7326908769746399
loss: 0.4154482992256389 ACC: 0.

train other predictors -- 2. BitterPredict (Adapt Boost)  in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5608695/

In [4]:

from contrast_predictors.BitterPredict import *
from hgnn.utils import bi_classify_metrics

qikprop_filepath = os.path.join(PWD+"/dataset/taste_dataset/qikprop.CSV") 
AdaBoost.train(bn_train_path, qikprop_filepath, "bn_Adaboost")


##### train this study predictors -- 3. gnns

In [25]:


from hgnn.load_data import load_data
from hgnn.features import Graph_smiles
from hgnn.trainers import train_gnn
from hgnn.model import HGNNPredictor
from contrast_gnns.GraphVAE import *
from contrast_gnns.GraphTransformers import *
from contrast_gnns.GraphSAGE import *
from contrast_gnns.WLN import *
from dgllife.model.model_zoo import AttentiveFPPredictor, PAGTNPredictor

n_tasks=2
n_node_feats=74
n_edge_feats=12

df = pd.read_csv(bn_train_path, header=0, sep='\t')
tuple_ls = list(zip(list(df['Smiles']), list(df['Label'])))
all = load_data(tuple_ls, featurizer=Graph_smiles, if_all=all, Stratify=False, if_torch=True, batchsize=int(len(tuple_ls)/16), graph=True, drop_last=False)
model = HGNNPredictor(
                    node_gat = True, 
                    edge_gat = True, 
                    weave = True, 
                    mpnn = True, 
                    n_node_feats=n_node_feats, n_edge_feats=n_edge_feats, num_layers=2, n_heads=5, n_hidden_feats=100, activation=F.relu, attn_activation=nn.LeakyReLU(negative_slope=0.2), attn_dropout=0, feat_dropout=0, xavier_normal=False, n_tasks=n_tasks,
                    )
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bn_hgnn.pth')

# other 5 models
model = WLNPredictor(node_in_feats=n_node_feats, edge_in_feats=n_edge_feats, node_out_feats=100, n_layers=2, project_in_feats=True, set_comparison=True, n_tasks=n_tasks)
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bn_wln.pth')

model = AttentiveFPPredictor(node_feat_size=n_node_feats,edge_feat_size=n_edge_feats,num_layers=2,num_timesteps=2,graph_feat_size=100,n_tasks=n_tasks,dropout=0.) 
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bn_afp.pth')

model = GraphSAGEPredictor(in_feats=n_node_feats, hidden_feats=[100,100], activation=None, dropout=None, aggregator_type=None, n_tasks=n_tasks)
rst = train_gnn.train_bi_classify_all(model, all=all, edge=False, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bn_gsage.pth')



####
df = pd.read_csv(bs_train_path, header=0, sep='\t')
tuple_ls = list(zip(list(df['Smiles']), list(df['Label'])))
all = load_data(tuple_ls, featurizer=Graph_smiles, if_all=all, Stratify=False, if_torch=True, batchsize=int(len(tuple_ls)/16), graph=True, drop_last=False)
model = HGNNPredictor(
                    node_gat = True, 
                    edge_gat = True, 
                    weave = True, 
                    mpnn = True, 
                    n_node_feats=n_node_feats, n_edge_feats=n_edge_feats, num_layers=2, n_heads=5, n_hidden_feats=100, activation=F.relu, attn_activation=nn.LeakyReLU(negative_slope=0.2), attn_dropout=0, feat_dropout=0, xavier_normal=False, n_tasks=n_tasks,
                    )
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bs_hgnn.pth')

# other 5 models
model = WLNPredictor(node_in_feats=n_node_feats, edge_in_feats=n_edge_feats, node_out_feats=100, n_layers=2, project_in_feats=True, set_comparison=True, n_tasks=n_tasks)
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bs_wln.pth')

model = AttentiveFPPredictor(node_feat_size=n_node_feats,edge_feat_size=n_edge_feats,num_layers=2,num_timesteps=2,graph_feat_size=100,n_tasks=n_tasks,dropout=0.) 
rst = train_gnn.train_bi_classify_all(model, all=all, edge=True, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bs_afp.pth')

model = GraphSAGEPredictor(in_feats=n_node_feats, hidden_feats=[100,100], activation=None, dropout=None, aggregator_type=None, n_tasks=n_tasks)
rst = train_gnn.train_bi_classify_all(model, all=all, edge=False, max_epochs=500, patience=7, save_folder=PWD+'/pretrained/',save_name='bs_gsage.pth')


/home/hy/Softwares/Program/miniconda3/envs/ml/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


loss: 0.6815344025106991 ACC: 0.6130467571644042 AUC: 0.5473659942325813
loss: 0.6384974051924313 ACC: 0.6417043740573152 AUC: 0.6127841298996419
loss: 0.6564112235518063 ACC: 0.6414781297134238 AUC: 0.6301215838370949
loss: 0.6489547456012053 ACC: 0.6296380090497737 AUC: 0.5854862605484447
loss: 0.6146537100567537 ACC: 0.6719457013574661 AUC: 0.6337969461099802
loss: 0.6235239646014046 ACC: 0.6601055806938159 AUC: 0.6328310197777924
loss: 0.5655158968532786 ACC: 0.7181749622926092 AUC: 0.6977843418409581
loss: 0.5266627730692134 ACC: 0.7499999999999998 AUC: 0.7333608723684465
loss: 0.4933238976141986 ACC: 0.782051282051282 AUC: 0.7610712616458644
loss: 0.5160002901273615 ACC: 0.7660633484162895 AUC: 0.7595965894205703
loss: 0.5117454791770262 ACC: 0.7547511312217194 AUC: 0.7518215849085687
loss: 0.5023986048558179 ACC: 0.779713423831071 AUC: 0.7641875124918304
loss: 0.4898134575170629 ACC: 0.7803167420814479 AUC: 0.7735145187419715
loss: 0.46182912763427286 ACC: 0.794419306184012 AUC:

/home/hy/Softwares/Program/miniconda3/envs/ml/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


loss: 0.6960928510217106 ACC: 0.5039583908680842 AUC: 0.49788787101938636
loss: 0.6474724376902861 ACC: 0.616266224799779 AUC: 0.6299315005075864
loss: 0.6110008078462937 ACC: 0.6859523151983797 AUC: 0.6816407044161599
loss: 0.5670797193751616 ACC: 0.7226364724293475 AUC: 0.7155764185075516
loss: 0.5445483414565816 ACC: 0.7316579213845162 AUC: 0.722517232827621
loss: 0.5186884017551646 ACC: 0.7508054865138545 AUC: 0.7497951473284459
loss: 0.5171804673531476 ACC: 0.7698609960416092 AUC: 0.7598204922752884
loss: 0.4864283975432901 ACC: 0.7863389487250299 AUC: 0.7857289330802991
loss: 0.474606734864852 ACC: 0.7831170026696124 AUC: 0.7767678889641039
loss: 0.4891859380637898 ACC: 0.7771794163674858 AUC: 0.7757402334543462
loss: 0.46751320011475506 ACC: 0.793059007640615 AUC: 0.7896142112472045
loss: 0.4464804337305181 ACC: 0.8071435146828685 AUC: 0.8073686875489366
loss: 0.4812052512870115 ACC: 0.7895148669796557 AUC: 0.7841266781196845
loss: 0.414044168065576 ACC: 0.8203995213108717 AUC: 

## Predictor Comparision on Bitter/Non-Bitter test dataset
+ CNN (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ descriptor MLP (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ RDKFP MLP (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ BitterPredict (https://www.nature.com/articles/s41598-017-12359-7)
+ BitterX (https://mdl.shsmu.edu.cn/BitterX)
+ BitterSweetForest/VirtualTaste (https://insilico-cyp.charite.de/VirtualTaste/)
+ HGNN
+ WLN
+ AttentiveFP
+ GraphSAGE

In [20]:
from contrast_predictors.BitterPredict import *
from hgnn.utils import bi_classify_metrics

qikprop_filepath = os.path.join(PWD+"/dataset/taste_dataset/qikprop.CSV") 

In [26]:
bn_rsts = []

model = BoCNN()
bn_test_imgs_dir = os.path.join(PWD, 'dataset/taste_dataset/bitter_nonbitter/bn_test_imgs/') 
if not os.path.exists(bn_test_imgs_dir):
    os.mkdir(bn_test_imgs_dir)
bn_cnn_train = cnn_prepare(bn_test, bn_test_imgs_dir, data_aug=False)
test = all_batchsize(bn_cnn_train, 10000)
rst = train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bn_cnn.pth',classnames=['Non-bitter','Bitter'])
print("CNN for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

bn_ds_mlp_test = Descriptors_bitter_nonbitter(bn_test)
test = all_batchsize(bn_ds_mlp_test, 10000)
model = BoMLP(n_feats=21)
rst =  train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bn_ds_mlp.pth',classnames=['Non-bitter','Bitter'])
print("MLP with Descriptors as input for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

bn_rdkfp_mlp_test = RDKFP(bn_test)
test = all_batchsize(bn_rdkfp_mlp_test, 10000)
model = BoMLP(n_feats=2048)
rst =  train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bn_rdkfp_mlp.pth',classnames=['Non-bitter','Bitter'])
print("MLP with RDKFP as input for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

trues,preds = AdaBoost.test(index_filepath=bn_test_path, qikprop_filepath=qikprop_filepath, save_name="bn_Adaboost")
rst = bi_classify_metrics(trues,preds,plot_cm=True,save_path=PWD+"/pretrained/bn_Adaboost.png", classnames=['Non-bitter','Bitter'])
print("BitterPredict based on Adaboost for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

from contrast_predictors.VirtualTaste import get_virtual_taste_result
preds = get_virtual_taste_result()
trues = list(pd.read_csv(bn_test_path, sep="\t",header=0)['Label'])
rst = bi_classify_metrics(np.array(trues), preds)
print("VisualTaste/BitterSweetForest based on RandomForest for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)



from hgnn.utils import simple_classify_metrics
from contrast_predictors.BitterX import get_result
preds = get_result()
trues = list(pd.read_csv(bn_test_path, sep="\t",header=0)['Label'])
rst = simple_classify_metrics(trues, preds)
print("BitterX for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)



df = pd.read_csv(bn_test_path, header=0, sep='\t')
tuple_ls = list(zip(list(df['Smiles']), list(df['Label'])))
test = load_data(tuple_ls, featurizer=Graph_smiles, if_all=all, Stratify=False, if_torch=True, batchsize=10000, graph=True, drop_last=False)
model = HGNNPredictor(
                    node_gat = True, 
                    edge_gat = True, 
                    weave = True, 
                    mpnn = True, 
                    n_node_feats=n_node_feats, n_edge_feats=n_edge_feats, num_layers=2, n_heads=5, n_hidden_feats=100, activation=F.relu, attn_activation=nn.LeakyReLU(negative_slope=0.2), attn_dropout=0, feat_dropout=0, xavier_normal=False, n_tasks=n_tasks,
                    )
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bn_hgnn.pth',classnames=['Non-bitter','Bitter'])
print("HGNN for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

model = WLNPredictor(node_in_feats=n_node_feats, edge_in_feats=n_edge_feats, node_out_feats=100, n_layers=2, project_in_feats=True, set_comparison=True, n_tasks=n_tasks)
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bn_wln.pth',classnames=['Non-bitter','Bitter'])
print("WLN for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

model = AttentiveFPPredictor(node_feat_size=n_node_feats,edge_feat_size=n_edge_feats,num_layers=2,num_timesteps=2,graph_feat_size=100,n_tasks=n_tasks,dropout=0.) 
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bn_afp.pth',classnames=['Non-bitter','Bitter'])
print("AFP for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

model = GraphSAGEPredictor(in_feats=n_node_feats, hidden_feats=[100,100], activation=None, dropout=None, aggregator_type=None, n_tasks=n_tasks)
rst = train_gnn.test_bi_classify(model, test=test, edge=False, save_path=PWD+'/pretrained/bn_gsage.pth',classnames=['Non-bitter','Bitter'])
print("GSAGE for Bitter/Non-bitter:")
print(rst)
bn_rsts.append(rst)

np.savetxt(PWD+"/results/compare_predictor/bitter_nonbitter" , bn_rsts, fmt='%.2f')


CNN for Bitter/Non-bitter:
[133.          30.          33.          82.           0.713
   0.816        0.732        0.773        0.64075473   0.722
   0.53116658   0.7644972 ]
MLP with Descriptors as input for Bitter/Non-bitter:
[128.          35.          28.          87.           0.757
   0.785        0.713        0.773        0.64020624   0.734
   0.53769718   0.77089891]
MLP with RDKFP as input for Bitter/Non-bitter:
[150.          13.          22.          93.           0.809
   0.92         0.877        0.874        0.78865269   0.842
   0.73912514   0.86447053]
BitterPredict based on Adaboost for Bitter/Non-bitter:
[133.          30.          32.          83.           0.722
   0.816        0.735        0.777        0.64523489   0.728
   0.53913004   0.76884503]
VisualTaste/BitterSweetForest based on RandomForest for Bitter/Non-bitter:
[95.         68.         20.         95.          0.826       0.583
  0.583       0.683       0.55340417  0.683       0.40890904  0.70445452]


/home/hy/Softwares/Program/miniconda3/envs/ml/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


HGNN for Bitter/Non-bitter:
[145.          18.          26.          89.           0.774
   0.89         0.832        0.842        0.73724724   0.802
   0.67155714   0.8317418 ]
WLN for Bitter/Non-bitter:
[146.          17.          23.          92.           0.8
   0.896        0.844        0.856        0.75796317   0.821
   0.7017971    0.84785276]
AFP for Bitter/Non-bitter:
[153.          10.          24.          91.           0.791
   0.939        0.901        0.878        0.79928832   0.842
   0.74746573   0.86497733]
GSAGE for Bitter/Non-bitter:
[133.          30.          17.          98.           0.852
   0.816        0.766        0.831        0.71359673   0.807
   0.66016078   0.83406242]


## Predictor Comparision on Bitter/Sweet test dataset
+ CNN (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ descriptor MLP (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ RDKFP MLP (https://www.sciencedirect.com/science/article/pii/S096399692200031X#f0005)
+ HGNN
+ WLN
+ AttentiveFP
+ GraphSAGE

In [27]:
bs_rsts = []

model = BoCNN()
bs_test_imgs_dir = os.path.join(PWD, 'dataset/taste_dataset/bitter_sweet/bs_test_imgs/') 
if not os.path.exists(bs_test_imgs_dir):
    os.mkdir(bs_test_imgs_dir)
bs_cnn_train = cnn_prepare(bs_test, bs_test_imgs_dir, data_aug=False)
test = all_batchsize(bs_cnn_train, 10000)
rst = train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bs_cnn.pth',classnames=['Sweet','Bitter'])
print("CNN for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

bs_ds_mlp_test = Descriptors_bitter_sweet(bs_test)
test = all_batchsize(bs_ds_mlp_test, 10000)
model = BoMLP(n_feats=17)
rst = train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bs_ds_mlp.pth',classnames=['Sweet','Bitter'])
print("MLP with Descriptors as input for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

bs_rdkfp_mlp_test = RDKFP(bs_test)
test = all_batchsize(bs_rdkfp_mlp_test, 10000)
model = BoMLP(n_feats=2048)
rst = train_mlp_cnn.test_bi_classify(model, test=test, plot_cm=True, save_path=PWD+'/pretrained/bs_rdkfp_mlp.pth',classnames=['Sweet','Bitter'])
print("MLP with RDKFP as input for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

df = pd.read_csv(bs_test_path, header=0, sep='\t')
tuple_ls = list(zip(list(df['Smiles']), list(df['Label'])))
test = load_data(tuple_ls, featurizer=Graph_smiles, if_all=all, Stratify=False, if_torch=True, batchsize=10000, graph=True, drop_last=False)
model = HGNNPredictor(
                    node_gat = True, 
                    edge_gat = True, 
                    weave = True, 
                    mpnn = True, 
                    n_node_feats=n_node_feats, n_edge_feats=n_edge_feats, num_layers=2, n_heads=5, n_hidden_feats=100, activation=F.relu, attn_activation=nn.LeakyReLU(negative_slope=0.2), attn_dropout=0, feat_dropout=0, xavier_normal=False, n_tasks=n_tasks,
                    )
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bs_hgnn.pth',classnames=['Sweet','Bitter'])
print("HGNN for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

# other 5 models
model = WLNPredictor(node_in_feats=n_node_feats, edge_in_feats=n_edge_feats, node_out_feats=100, n_layers=2, project_in_feats=True, set_comparison=True, n_tasks=n_tasks)
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bs_wln.pth',classnames=['Sweet','Bitter'])
print("WLN for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

model = AttentiveFPPredictor(node_feat_size=n_node_feats,edge_feat_size=n_edge_feats,num_layers=2,num_timesteps=2,graph_feat_size=100,n_tasks=n_tasks,dropout=0.) 
rst = train_gnn.test_bi_classify(model, test=test, edge=True, save_path=PWD+'/pretrained/bs_afp.pth',classnames=['Sweet','Bitter'])
print("AFP for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)

model = GraphSAGEPredictor(in_feats=n_node_feats, hidden_feats=[100,100], activation=None, dropout=None, aggregator_type=None, n_tasks=n_tasks)
rst = train_gnn.test_bi_classify(model, test=test, edge=False, save_path=PWD+'/pretrained/bs_gsage.pth',classnames=['Sweet','Bitter'])
print("GSAGE for Bitter/Sweet:")
print(rst)
bs_rsts.append(rst)


np.savetxt(PWD+"/results/compare_predictor/bitter_sweet" , bs_rsts, fmt='%.2f')

CNN for Bitter/Sweet:
[121.          18.          31.          84.           0.73
   0.871        0.824        0.807        0.72358177   0.774
   0.61018901   0.80046919]
MLP with Descriptors as input for Bitter/Sweet:
[88.         51.         25.         90.          0.783       0.633
  0.638       0.701       0.59796266  0.703       0.41638002  0.70785111]
MLP with RDKFP as input for Bitter/Sweet:
[124.          15.          22.          93.           0.809
   0.892        0.861        0.854        0.78299098   0.834
   0.7055876    0.85039099]


/home/hy/Softwares/Program/miniconda3/envs/ml/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


HGNN for Bitter/Sweet:
[126.          13.          18.          97.           0.843
   0.906        0.882        0.878        0.81466061   0.862
   0.75337781   0.87497654]
WLN for Bitter/Sweet:
[125.          14.          20.          95.           0.826
   0.899        0.872        0.866        0.7987242    0.848
   0.72948637   0.86268377]
AFP for Bitter/Sweet:
[133.           6.          35.          80.           0.696
   0.957        0.93         0.839        0.78491358   0.796
   0.686316     0.82624335]
GSAGE for Bitter/Sweet:
[124.          15.          18.          97.           0.843
   0.892        0.866        0.87         0.80137856   0.854
   0.73743535   0.8677823 ]
